In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import logging
from psutil import virtual_memory

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
ram_gb = virtual_memory().total / 1e9

In [ ]:
tf_response = {
    'error': None,
    'TF version': '',
    'COLAB': None,
    'GPU': False,
    'ram_gb': ''
}

In [ ]:
try:
    # drive
    from google.colab import drive
    IN_COLAB = 'google.colab' in sys.modules

    # updating tensorflow version
    %tensorflow_version 2.x

    # tensorflow-gpu & tensorflowjs
    !pip install tensorflow-gpu # !pip install tensorflow_text # I could use BERT
    !pip install tensorflowjs

    # NLP (nltk, stanza, spacy)
    !pip install nltk 
    !pip install stanza
    !pip install spacy
    !spacy download en_core_web_sm # sm md lg
    !python -m spacy download en
except OSError as error:
    # debugging error
    response['error'] = logging.debug('You are not using your specify version of TensorFlow')
    IN_COLAB = False

    # install requirements
    !pip install -r '../requirements.txt'
finally:
    tf_response['COLAB'] = IN_COLAB
    
    # Importing tensroflow core
    import tensorflow as tf
    import tensorflowjs as tfjs
    from tensorflow import keras
    from keras.utils import to_categorical
    from sklearn import preprocessing

    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from keras.callbacks import EarlyStopping
    from keras.callbacks import ModelCheckpoint

    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional, GlobalMaxPool1D

    from sklearn.model_selection import train_test_split
    
    # GPU and RAM response
    if tf.config.list_physical_devices('GPU'):
        GPU = True
        tf_response['GPU'] = GPU
        tf_response['TF_version'] = tf.__version__
        
        if tf_response['COLAB'] == True:
            if gpu_info.find('failed') >= 0:
                print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator')
                print('Re-execute this cell.')
            else:
                print(gpu_info)
            
            if ram_gb < 20:
                print('To enable a high-RAM runtime, select the Runtime > "Change runtime type menu"')
                print('Select high-RAM in the runtime shape dropdown')
                print('Re-execute this cell')
                tf_response['ram_gb'] = 'low-RAM runtime'
            else:
                tf_response['ram_gb'] = 'high-RAM runtime'
            print('\nRuntime {:.2f} GB of available RAM\n'.format(ram_gb))

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
tf_response

{'COLAB': True, 'GPU': False, 'TF version': '', 'error': None, 'ram_gb': ''}

In [ ]:
# Data analysis
from keras.layers.merge import Concatenate
from collections import Counter
import matplotlib.pyplot as plt
from keras.layers import Input
from keras.models import Model
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.utils import plot_model
import pandas as pd
import numpy as np
import sys
import re

%matplotlib inline

# Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator # to create a Word Cloud
from PIL import Image # Pillow with WordCloud to image manipulation

In [ ]:
# nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Spacy NLP
import spacy
spNLP = spacy.load('en_core_web_sm')
spNLP.max_length = 103950039 # or higher
# spacy.prefer_gpu() #will not work witczh stanza

In [ ]:
def nltk_lemma(text):
    lemmatizer = WordNetLemmatizer()
    lemmatizer.lemmatize(text)

## **Global Variable**

In [ ]:
maxlen = 30
epochs = 30
batch_size=16

In [ ]:
def load_clean_dataset():
  
    !mkdir -p datasets
    !wget -nc https://raw.githubusercontent.com/Y4rd13/sentiment-analysis/master/datasets/results/categories_dataset.csv -P datasets
    df_cat = pd.read_csv('./datasets/categories_dataset.csv', encoding='utf-8')

    df_cat.drop_duplicates(inplace=True)
    df_cat = df_cat.loc[:, ~df_cat.columns.str.contains('^Unnamed')]

    df = df_cat[['word','Valence','Arousal','Dominance','category']]
    df.dropna(inplace=True)
    
    y = df['category']
    X  = df[['word','Valence','Arousal','Dominance']]

    return X, y

In [ ]:
def preprocessing(X, y, maxlen):
  
  from sklearn import preprocessing
  # The X variable contains the feature set, where as the y variable contains label set.

  # label_encoder object knows how to understand word labels.
  label_encoder = preprocessing.LabelEncoder()

  # Encode labels in column 'category'.
  y = label_encoder.fit_transform(y)

  #splitting the dataset into train and test
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

  #convert to categorical variable
  y_test_labels = y_test
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  #only text data
  X1_train =  X_train['word']
  X1_test =  X_test['word']
  

  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(X1_train)

  X1_train = tokenizer.texts_to_sequences(X1_train)
  X1_test = tokenizer.texts_to_sequences(X1_test)

  vocab_size = len(tokenizer.word_index) + 1

  X1_train = pad_sequences(X1_train, padding='post', maxlen=maxlen)
  X1_test = pad_sequences(X1_test, padding='post', maxlen=maxlen)

  #now we are taking numerical value

  X2_train = X_train[['Valence','Arousal','Dominance']].values
  X2_test = X_test[['Valence','Arousal','Dominance']].values

  return X1_train,y_train,y_test, X1_test,X2_train,X2_test,maxlen,vocab_size,tokenizer, y_test_labels

GloVe word embeddings for creating word vectors

In [ ]:
def GloVe(vocab_size,tokenizer):

  embeddings_dictionary = dict()

  glove_file = open('/content/drive/My Drive/glove.6B.100d.txt', encoding="utf8")

  for line in glove_file:
      records = line.split()
      word = records[0]
      vector_dimensions = asarray(records[1:], dtype='float32')
      embeddings_dictionary[word] = vector_dimensions

  glove_file.close()

  embedding_matrix = zeros((vocab_size, 100))
  for word, index in tokenizer.word_index.items():
      embedding_vector = embeddings_dictionary.get(word)
      if embedding_vector is not None:
          embedding_matrix[index] = embedding_vector
  
  return embedding_matrix

In [ ]:
def build_model(embedding_matrix,maxlen,vocab_size):

  input_1 = Input(shape=(maxlen,)) #for text, The first submodel will accept textual input
  input_2 = Input(shape=(3,)) #for number

  embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(input_1)
  LSTM_Layer_1 = LSTM(128)(embedding_layer) #This submodel will consist of an input shape layer, an embedding layer, and an LSTM layer of 128 neurons. 

  dense_layer_1 = Dense(10, activation='relu')(input_2)
  dense_layer_2 = Dense(10, activation='relu')(dense_layer_1)

  concat_layer = Concatenate()([LSTM_Layer_1, dense_layer_2])
  dense_layer_3 = Dense(10, activation='relu')(concat_layer)
  output = Dense(10, activation='softmax')(dense_layer_3)
  model = Model(inputs=[input_1, input_2], outputs=output)

  model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=['acc'])
  model.summary()

  return model

# Train model

In [ ]:
def train(model,X1_train,X2_train,y_train,batch_size,epochs):

  es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode = 'min', patience=5)
  mc = tf.keras.callbacks.ModelCheckpoint('tfjsmodel.h5', monitor='val_acc',mode = 'max', verbose=1, save_best_only=True)

  history = model.fit(x=[X1_train, X2_train], y=y_train, batch_size=batch_size, callbacks = [mc, es],epochs=epochs, verbose=1, validation_split=0.15)

  return history

### **Plotting history**

In [ ]:

def plot_history_(history,score):

  print("Test Score:", score[0])
  print("Test Accuracy:", score[1])

  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])

  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train','test'], loc='upper left')
  plt.show()
  plt.savefig('Accuracy.png')

  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])

  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train','test'], loc='upper left')
  plt.show()
  plt.savefig('Loss.png')

## **Main**

In [ ]:
X, y = load_clean_dataset() #load and clean dataset

X1_train,y_train,y_test, X1_test,X2_train,X2_test,maxlen,vocab_size,tokenizer, y_test_labels = preprocessing(X, y, maxlen) #preprocess dataset

embedding_matrix = GloVe(vocab_size, tokenizer) #glove

model = build_model(embedding_matrix, maxlen,vocab_size) #building model

plot_model(model, to_file='model_plot3.png', show_shapes=True, show_layer_names=True)

history = train(model,X1_train,X2_train,y_train, batch_size,epochs)

File ‘datasets/categories_dataset.csv’ already there; not retrieving.



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 100)      1998900     input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 10)           40          input_2[0][0]                    
_______________________________________________________________________________________

In [ ]:
from keras.models import load_model

In [ ]:
test_model = load_model('tfjsmodel.h5')
prediction = test_model.predict(x=[X1_test,X2_test], verbose = 1)

421/421 [==============================] - 10s 24ms/step


In [ ]:
prediction

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 2.5040624e-01,
        1.4474839e-15, 3.3088673e-10],
       [1.5616773e-06, 3.1854683e-03, 7.9136451e-05, ..., 4.1300807e-09,
        2.0041482e-09, 5.0861394e-04],
       [5.2709973e-01, 4.5314166e-01, 4.5239049e-04, ..., 6.5060147e-14,
        1.5599744e-12, 1.1741150e-08],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 2.1620320e-02,
        3.2118027e-18, 1.3074522e-10],
       [1.2559842e-06, 2.5010994e-03, 5.9200596e-05, ..., 3.2649863e-09,
        2.1630269e-09, 4.2986666e-04],
       [4.6946737e-01, 4.2423484e-01, 1.3772710e-04, ..., 1.1460935e-14,
        3.8080537e-12, 3.5124068e-09]], dtype=float32)

In [ ]:
y_classes = prediction.argmax(axis=-1)

In [ ]:
y_classes.shape

(13472,)

In [ ]:
test_score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

421/421 [==============================] - 6s 13ms/step - loss: 0.5099 - acc: 0.6644


In [ ]:
y_test_labels.shape

(13472,)

In [ ]:
import pandas as pd
df = pd.DataFrame(data = prediction[0:, 0:], 
                  index = [str(i) for i in y_test_labels],
                  columns = [str(i) for i in range(prediction.shape[1])])

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9
7,0.000000e+00,0.000000e+00,0.000000e+00,2.814949e-34,5.272422e-01,1.872099e-21,2.223515e-01,2.504062e-01,1.447484e-15,3.308867e-10
3,1.561677e-06,3.185468e-03,7.913645e-05,9.777549e-01,8.974229e-09,1.847034e-02,1.210822e-13,4.130081e-09,2.004148e-09,5.086139e-04
1,5.270997e-01,4.531417e-01,4.523905e-04,5.446275e-04,1.713505e-13,1.876152e-02,2.591948e-19,6.506015e-14,1.559974e-12,1.174115e-08
4,2.429084e-27,6.693740e-23,5.168884e-26,9.710953e-12,9.553616e-01,7.918949e-05,9.747206e-04,4.112374e-02,3.637110e-06,2.457073e-03
4,0.000000e+00,0.000000e+00,0.000000e+00,1.378620e-31,5.579849e-01,4.962835e-18,1.076320e-01,3.343830e-01,2.123751e-11,1.936470e-10
...,...,...,...,...,...,...,...,...,...,...
6,0.000000e+00,0.000000e+00,0.000000e+00,1.905152e-34,3.784591e-01,3.634475e-22,3.138541e-01,3.076868e-01,5.721536e-16,3.414659e-10
4,0.000000e+00,0.000000e+00,0.000000e+00,9.432897e-31,5.380160e-01,3.673253e-14,2.336550e-02,4.385383e-01,8.029237e-05,1.821004e-12
4,0.000000e+00,0.000000e+00,0.000000e+00,4.739637e-36,9.410002e-01,7.464538e-24,3.737944e-02,2.162032e-02,3.211803e-18,1.307452e-10
3,1.255984e-06,2.501099e-03,5.920060e-05,9.731141e-01,8.090341e-09,2.389454e-02,1.014432e-13,3.264986e-09,2.163027e-09,4.298667e-04


In [ ]:
df = df.T
df

,7,3,1,4,4,6,0,7,7,1,9,6,4,7,5,2,7,6,9,7,7,7,7,7,9,2,4,4,3,1,4,4,4,7,7,7,2,1,1,4,...,1,0,9,4,7,8,7,4,6,6,2,5,7,4,4,3,4,6,7,7,3,4,4,4,7,8,3,2,2,2,8,1,7,5,4,6,4,4,3,0
0,0.000000e+00,1.561677e-06,5.270997e-01,2.429084e-27,0.000000e+00,0.000000e+00,5.158160e-01,0.000000e+00,0.000000e+00,3.735165e-05,3.668198e-17,0.000000e+00,0.000000e+00,0.000000e+00,2.776904e-09,3.651611e-05,0.000000e+00,0.000000e+00,3.144307e-17,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.576615e-17,3.637183e-05,0.000000e+00,1.513411e-29,5.311974e-07,5.130428e-01,0.000000e+00,0.000000e+00,8.653192e-37,0.000000e+00,0.000000e+00,0.000000e+00,4.363348e-05,5.380855e-01,4.314213e-05,0.000000e+00,...,3.891398e-05,4.709960e-01,4.038349e-17,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.898001e-05,2.880606e-15,0.000000e+00,0.000000e+00,0.000000e+00,1.340457e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.340457e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.799407e-06,3.667599e-05,4.382597e-05,3.732224e-05,0.000000e+00,4.277032e-05,0.000000e+00,1.559290e-09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.255984e-06,4.694674e-01
1,0.000000e+00,3.185468e-03,4.531417e-01,6.693740e-23,0.000000e+00,0.000000e+00,4.696563e-01,0.000000e+00,0.000000e+00,5.757914e-01,1.169106e-09,0.000000e+00,0.000000e+00,0.000000e+00,4.007520e-10,5.677760e-01,0.000000e+00,0.000000e+00,9.416806e-10,0.000000e+00,1.253052e-38,0.000000e+00,0.000000e+00,0.000000e+00,1.167662e-09,5.615577e-01,0.000000e+00,1.028161e-24,1.126621e-03,4.704629e-01,0.000000e+00,0.000000e+00,2.768751e-33,0.000000e+00,1.759759e-38,0.000000e+00,5.632722e-01,4.281384e-01,6.798029e-01,0.000000e+00,...,5.615543e-01,4.243913e-01,1.240998e-09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.546896e-01,1.618275e-13,0.000000e+00,0.000000e+00,0.000000e+00,2.684672e-03,0.000000e+00,0.000000e+00,0.000000e+00,1.336187e-38,2.684672e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.516747e-03,5.800095e-01,5.571595e-01,6.102787e-01,0.000000e+00,5.612892e-01,0.000000e+00,2.097847e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.501099e-03,4.242348e-01
2,0.000000e+00,7.913645e-05,4.523905e-04,5.168884e-26,0.000000e+00,0.000000e+00,6.053488e-04,0.000000e+00,0.000000e+00,4.222555e-01,4.324693e-09,0.000000e+00,0.000000e+00,0.000000e+00,3.145879e-15,4.305170e-01,4.592374e-38,0.000000e+00,2.857742e-09,0.000000e+00,7.959877e-38,0.000000e+00,9.300118e-38,0.000000e+00,4.314297e-09,4.368617e-01,0.000000e+00,1.149106e-27,1.700986e-05,5.614549e-04,0.000000e+00,0.000000e+00,1.078333e-37,8.614149e-38,6.239887e-38,0.000000e+00,4.353651e-01,2.909722e-04,3.172506e-01,0.000000e+00,...,4.370492e-01,1.385848e-04,4.814291e-09,0.000000e+00,9.534098e-38,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.440430e-01,1.601549e-17,7.935331e-38,0.000000e+00,0.000000e+00,6.431768e-05,0.000000e+00,0.000000e+00,0.000000e+00,1.260890e-37,6.431768e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.780314e-05,4.177130e-01,4.415888e-01,3.868808e-01,0.000000e+00,4.373572e-01,8.311837e-38,2.103068e-10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.920060e-05,1.377271e-04
3,2.814949e-34,9.777549e-01,5.446275e-04,9.710953e-12,1.378620e-31,2.456891e-34,5.679999e-04,1.221200e-34,1.253603e-30,1.865577e-03,2.302184e-04,1.164473e-34,1.550465e-34,2.201292e-34,1.337115e-05,1.624050e-03,5.513233e-24,1.924473e-34,2.608823e-04,8.395438e-31,2.091467e-23,2.388064e-34,7.948469e-24,1.819328e-34,2.338389e-04,1.499449e-03,5.534067e-36,4.780757e-13,8.983201e-01,5.565775e-04,3.395728e-26,9.561129e-32,7.087181e-19,5.906288e-24,1.444868e-23,1.557755e-34,1.273835e-03,5.138168e-04,2.859377e-03,1.670063e-36,...,1.312797e-03,3.646803e-04,2.099486e-04,1.670724e-34,8.394828e-24,3.187096e-31,3.393590e-34,2.678049e-34,1.720542e-34,2.267634e-34,1.185494e-03,3.934209e-06,9.142500e-24,2.

In [ ]:
# keep only 10 rows from dataframe which contain the label names with highest probability values
df_10_classes = pd.DataFrame()
for col in range(df.shape[1]):
  df_col = df.iloc[:,col]
  df_max = df_col.nlargest(1)
  df_max_index = df_max.index
  df_10_classes[col] = df_max_inde
  x

# rename columns to labels of each video
df_10_classes.columns = y_test_labels
df_10_classes

,7,3,1,4,4,6,0,7,7,1,9,6,4,7,5,2,7,6,9,7,7,7,7,7,9,2,4,4,3,1,4,4,4,7,7,7,2,1,1,4,...,1,0,9,4,7,8,7,4,6,6,2,5,7,4,4,3,4,6,7,7,3,4,4,4,7,8,3,2,2,2,8,1,7,5,4,6,4,4,3,0
0,4,3,0,4,4,4,0,4,4,1,9,4,4,4,5,1,7,4,9,4,7,4,7,4,9,1,4,4,3,0,4,7,4,7,7,4,1,0,1,4,...,1,0,9,4,7,8,4,4,4,4,1,5,7,4,4,3,4,4,4,7,3,4,4,4,4,8,3,1,1,1,8,1,7,5,7,4,4,4,3,0


In [ ]:
y_test_labels

array([7, 3, 1, ..., 4, 3, 0])

In [ ]:
#decoding the predicted value
label_dict = { 0 : 'confident',
               1 : 'unconfident',
               2 : 'pos_hp', 
               3 : 'neg_hp', 
               4 : 'interested',
               5 : 'uninterested', 
               6 : 'happy', 
               7 : 'unhappy',
               8 : 'friendly',
               9 : 'unfriendly'}

In [ ]:
df_pred_new = pd.DataFrame(data=y_test_labels, columns=['Predicted Value'])

In [ ]:
df_pred_new['Predicted Category'] = df_pred_new['Predicted Value'].map(label_dict)

In [ ]:
df_pred_new

,Predicted Value,Predicted Category
0,7,unhappy
1,3,neg_hp
2,1,unconfident
3,4,interested
4,4,interested
...,...,...
13467,6,happy
13468,4,interested
13469,4,interested
13470,3,neg_hp
